In [2]:
import os
import sys
import h5py
import torch
import warnings
import torch.nn.functional as F

from os.path import join as pjoin
from models.quantized_deep4 import QuantDeep4Net
from braindecode.models.deep4 import Deep4Net
from braindecode.torch_ext.optimizers import AdamW
from braindecode.torch_ext.util import set_random_seeds

In [7]:
datapath = "./dataset/KU_mi_smt.h5"
set_random_seeds(seed=20200205, cuda=False)
n_classes = 2

In [8]:
dfile = h5py.File(datapath, 'r')

In [9]:
def get_data(subj):
    dpath = '/s' + str(subj)
    X = dfile[pjoin(dpath, 'X')][:]
    Y = dfile[pjoin(dpath, 'Y')][:]
    return X, Y

In [ ]:
subj = 3
X, Y = get_data(subj)
X_train, Y_train = X[:200], Y[:200]
X_val, Y_val = X[200:300], Y[200:300]
X_test, Y_test = X[300:], Y[300:]

suffix = 's' + str(subj)
n_classes = 2
in_chans = X.shape[1]

# final_conv_length = auto ensures we only get a single output in the time dimension
model = Deep4Net(in_chans=in_chans, n_classes=n_classes,
                 input_time_length=X.shape[2],
                 final_conv_length='auto')

# these are good values for the deep model
optimizer = AdamW(model.parameters(), lr=1 * 0.01, weight_decay=0.5*0.001)
model.compile(loss=F.nll_loss, optimizer=optimizer, iterator_seed=1, )

model.fit(X_train, Y_train, epochs=200, batch_size=16, scheduler='cosine',
          validation_data=(X_val, Y_val), remember_best_column='valid_loss')

test_loss = model.evaluate(X_test, Y_test)
# model.epochs_df.to_csv(pjoin(outpath, 'epochs_' + suffix + '.csv'))
# with open(pjoin(outpath, 'test_subj_' + str(subj) + '.json'), 'w') as f:
#     json.dump(test_loss, f)

In [14]:
print(test_loss)

{'loss': 0.5699537992477417, 'misclass': 0.31000000000000005, 'runtime': 0.0003535747528076172}


In [ ]:
for subj in subjs:
    # Get data for within-subject classification
    X, Y = get_data(subj)
    X_train, Y_train = X[:200], Y[:200]
    X_val, Y_val = X[200:300], Y[200:300]
    X_test, Y_test = X[300:], Y[300:]

    suffix = 's' + str(subj)
    n_classes = 2
    in_chans = X.shape[1]

    # final_conv_length = auto ensures we only get a single output in the time dimension
    model = Deep4Net(in_chans=in_chans, n_classes=n_classes,
                     input_time_length=X.shape[2],
                     final_conv_length='auto').cuda()

    # these are good values for the deep model
    optimizer = AdamW(model.parameters(), lr=1 * 0.01, weight_decay=0.5*0.001)
    model.compile(loss=F.nll_loss, optimizer=optimizer, iterator_seed=1, )

    model.fit(X_train, Y_train, epochs=200, batch_size=16, scheduler='cosine',
              validation_data=(X_val, Y_val), remember_best_column='valid_loss')

    test_loss = model.evaluate(X_test, Y_test)
    model.epochs_df.to_csv(pjoin(outpath, 'epochs_' + suffix + '.csv'))
    with open(pjoin(outpath, 'test_subj_' + str(subj) + '.json'), 'w') as f:
        json.dump(test_loss, f)


In [ ]:
# dfile.close()